In [2]:
import sys

In [3]:
import torch
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-4B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype="auto",
    device_map="cuda"
)

D:\conda_env\vllm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 3/3 [00:09<00:00,  3.33s/it]


In [15]:
# prepare the model input
prompt = "中文回答 你是什么"
messages = [
    {"role": "user", "content": prompt}
]

#apply_chat_template 函数本身是通用的
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,#表示 apply_chat_template 返回的是格式化好的字符串，而不是直接返回 token IDs。
    add_generation_prompt=True,#这会在格式化好的对话字符串末尾添加一个特殊的 token（例如 \n<|assistant|>\n 或类似的），告诉模型“轮到你了，请开始生成回答”。
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)

In [17]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device) #将前面生成的格式化字符串 text 进行分词，转换成模型可以处理的 token IDs。
#return_tensors="pt" 指定返回 PyTorch tensors。
# model_inputs现在是一个字典，里面包含 input_ids 和 attention_mask 等，这是模型直接的输入

In [23]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)#首先，它会生成“思考”部分.中间用think分割,然后才是真正的答案
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()#只保留模型生成的新 token（即从输入结束位置之后的部分）。

In [4]:
# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)#到模型输出中第一个 </think>（token ID 151668）的准确位置。
except ValueError:
    index = 0
#可能 enable_thinking 没有生效，或者模型没有生成思考部分

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

thinking content: <think>
嗯，用户问“你是什么”，我需要回答这个问题。首先，我需要确定用户的需求是什么。可能他们想了解我的身份或者功能。我应该从基本的介绍开始，比如我是通义千问，阿里巴巴集团旗下的语言模型。

接下来，用户可能想知道我的用途，比如能否回答问题、创作内容、编程等。需要简要说明我的能力范围，比如多语言支持、逻辑推理、创造性和开放式对话。

然后，用户可能有更深层的需求，比如他们可能在寻找一个可靠的助手，或者想了解我的技术背景。需要提到我是基于大规模数据训练的，能够处理各种任务，但也要注意不要过度承诺，比如不能保证所有问题都能回答。

另外，用户可能有潜在的问题，比如隐私和安全，需要说明我的设计原则，比如遵循法律法规，保护用户隐私。

最后，保持回答简洁明了，同时提供足够的信息让用户了解我的能力和用途。可能需要用分点的方式，但用户要求中文回答，所以保持自然的口语化表达。
</think>
content: 我是通义千问，阿里巴巴集团旗下的大型语言模型，我能够帮助您回答问题、创作内容、进行编程等。我的设计目标是提供可靠、安全、高效的对话体验，同时遵循法律法规和伦理规范。如果您有任何问题或需要帮助，欢迎随时告诉我！


In [7]:
 output_ids

[151667,
 198,
 106287,
 3837,
 20002,
 56007,
 2073,
 56568,
 102021,
 33590,
 35946,
 85106,
 102104,
 105073,
 1773,
 101140,
 3837,
 35946,
 85106,
 60610,
 20002,
 99172,
 99794,
 99245,
 1773,
 87267,
 99650,
 109623,
 97611,
 101294,
 5373,
 98380,
 3837,
 100631,
 97611,
 105795,
 1773,
 100622,
 31935,
 64559,
 99320,
 56007,
 3837,
 35946,
 101909,
 101951,
 102064,
 104949,
 3837,
 73670,
 102104,
 100646,
 86119,
 3837,
 104223,
 108704,
 3837,
 71817,
 105051,
 49567,
 3407,
 104326,
 3837,
 35946,
 85106,
 101118,
 20002,
 87267,
 9370,
 102193,
 1773,
 104560,
 100714,
 20002,
 3837,
 99172,
 99794,
 97611,
 99788,
 24968,
 74763,
 104560,
 113129,
 3837,
 99172,
 99794,
 97611,
 99361,
 104449,
 1773,
 100632,
 3837,
 86119,
 99792,
 98237,
 99534,
 3837,
 87267,
 100009,
 100714,
 20002,
 99172,
 81167,
 97611,
 101294,
 3407,
 101889,
 3837,
 35946,
 85106,
 103944,
 102104,
 102188,
 100136,
 110485,
 1773,
 85106,
 66394,
 104198,
 107076,
 100338,
 111477,
 31935,
